In [ ]:
#install keras and tensorflow 
!pip install tensorflow==2.5.0
!pip install keras==2.4.3


import os
import time
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Assign which GPU to run on, "0" or "1"
import tensorflow as tf
import scipy.io as sio
from tensorflow import keras
tf.keras.backend.clear_session()
import numpy as np
from tensorflow.python.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPool2D, UpSampling2D, concatenate, Flatten, Reshape, Cropping2D
from tensorflow.python.keras.models import Model
import matplotlib.pyplot as plt


from google.colab import drive
drive.mount('/content/gdrive')
# !unzip "/content/training_data.zip" -d "/content"

!nvidia-smi
tf.test.gpu_device_name()


Training_Dataset_path_centroid = "/content/gdrive/MyDrive/MPhys Project/centroid/run_27/centroid.mat"
Training_Dataset_path_gaussfilt = "/content/gdrive/MyDrive/MPhys Project/centroid/run_27/gaussfilt.mat"
checkpoint_path = "/content/gdrive/MyDrive/MPhys Project/centroid/run_27"


# set the number of epochs
Nepochs = 500

# load the training data
X = sio.loadmat(Training_Dataset_path_centroid)
X = X['images_sampled_mat']
# X = X['single_image']
X = np.swapaxes(X, 0, -1)
X = np.swapaxes(X, 1, 2)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))

X_blur = sio.loadmat(Training_Dataset_path_gaussfilt)
X_blur = X_blur['images_gaussfilt_mat']

X_blur = np.swapaxes(X_blur, 0, -1)
X_blur = np.swapaxes(X_blur, 1, 2)
X_blur = np.reshape(X_blur, (X.shape[0], X.shape[1], X.shape[2], 1))


# set training/validation split
X = X[:1500]
X_blur = X_blur[:1500]
X_training = X[150:]
X_training_blur = X_blur[150:]
X_validation = X[:150]
X_validation_blur = X_blur[:150]


# model
def create_model():
    UB = True
    w = 5
    inputs = Input((48,48,1))
    conv1 = Conv2D(64, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(inputs)
    pool1 = MaxPool2D((2,2), padding='same')(conv1)

    conv2 = Conv2D(128, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool1)
    pool2 = MaxPool2D((2,2), padding='same')(conv2)

    conv3 = Conv2D(256, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool2)
    pool3 = MaxPool2D((2,2), padding='same')(conv3)

    conv4 = Conv2D(512, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool3)
    pool4 = MaxPool2D((2,2), padding='same')(conv4)

    conv5 = Conv2D(1024, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool4)

    deconv4 = Conv2DTranspose(512, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(conv5)
    merge4 = concatenate([conv4,deconv4], axis = 3)

    deconv3 = Conv2DTranspose(256, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge4)
    merge3 = concatenate([conv3,deconv3], axis = 3)


    deconv2 = Conv2DTranspose(128, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge3)
    merge2 = concatenate([conv2,deconv2], axis = 3)

    deconv1 = Conv2DTranspose(64, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge2)
    merge1 = concatenate([conv1,deconv1], axis = 3)

    conv_last = Conv2D(1, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(merge1)

    model = Model(inputs,conv_last)

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.00010), loss = "mean_squared_error", metrics = ["accuracy"])
    
    return model

model = create_model()

model.summary()


checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq = 250,
                                                 verbose = 1)

start_time = time.time()
history = model.fit(X_training_blur, X_training, validation_data = (X_validation_blur, X_validation), epochs = Nepochs, batch_size = 32, callbacks=[cp_callback])

training_time = time.time()-start_time

# test and save on validation data
X_val_blur = X_validation_blur
X_val = X_validation
# model.load_weights(checkpoint_path)
start_time = time.time()
predict = model.predict(X_val_blur)
testing_time = time.time()-start_time
dictionary = {}
dictionary['results'] = {'input':X_val_blur, 'predictions':predict, 'validation':X_val}
dictionary['time'] = {'training_time': training_time, 'testing_time': testing_time}

sio.savemat("/content/gdrive/MyDrive/MPhys Project/centroid/run_27/results.mat", dictionary)

In [ ]:
#install keras and tensorflow 
!pip install tensorflow==2.5.0
!pip install keras==2.4.3


# !pip install tensorflow==2.1.0
# !pip install keras==2.2.4

import os
import time
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Assign which GPU to run on, "0" or "1"
import tensorflow as tf
import scipy.io as sio
from tensorflow import keras
tf.keras.backend.clear_session()
import numpy as np
from tensorflow.python.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPool2D, UpSampling2D, concatenate, Flatten, Reshape, Cropping2D
from tensorflow.python.keras.models import Model

from google.colab import drive
drive.mount('/content/gdrive')
# !unzip "/content/training_data.zip" -d "/content"

!nvidia-smi
tf.test.gpu_device_name()


Training_Dataset_path_centroid = "/content/gdrive/MyDrive/MPhys Project/centroid/run_27/centroid.mat"
Training_Dataset_path_gaussfilt = "/content/gdrive/MyDrive/MPhys Project/centroid/run_27/gaussfilt.mat"
checkpoint_path = "/content/gdrive/MyDrive/MPhys Project/centroid/run_27"


Nepochs = 500

X = sio.loadmat(Training_Dataset_path_centroid)
X = X['images_sampled_mat']
X = np.swapaxes(X, 0, -1)
X = np.swapaxes(X, 1, 2)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))

X_blur = sio.loadmat(Training_Dataset_path_gaussfilt)
X_blur = X_blur['images_gaussfilt_mat']

X_blur = np.swapaxes(X_blur, 0, -1)
X_blur = np.swapaxes(X_blur, 1, 2)
X_blur = np.reshape(X_blur, (X.shape[0], X.shape[1], X.shape[2], 1))


X = X[:1500]
X_blur = X_blur[:1500]
X_training = X[150:]
X_training_blur = X_blur[150:]
X_validation = X[:150]
X_validation_blur = X_blur[:150]


# model
def create_model():
    UB = True
    w = 5
    inputs = Input((48,48,1))
    conv1 = Conv2D(64, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(inputs)
    pool1 = MaxPool2D((2,2), padding='same')(conv1)

    conv2 = Conv2D(128, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool1)
    pool2 = MaxPool2D((2,2), padding='same')(conv2)

    conv3 = Conv2D(256, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool2)
    pool3 = MaxPool2D((2,2), padding='same')(conv3)

    conv4 = Conv2D(512, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool3)
    pool4 = MaxPool2D((2,2), padding='same')(conv4)

    conv5 = Conv2D(1024, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(pool4)


    deconv4 = Conv2DTranspose(512, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(conv5)
    # cropped_decon4 = Cropping2D(cropping=((1, 0), (1, 0)))(deconv4)
    # merge4 = concatenate([conv4,cropped_decon4], axis = 3)
    merge4 = concatenate([conv4,deconv4], axis = 3)

    deconv3 = Conv2DTranspose(256, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge4)
    # cropped_deconv3 = Cropping2D(cropping=((1, 0), (0, 0)))(deconv3)

    # merge3 = concatenate([conv3,cropped_deconv3], axis = 3)
    merge3 = concatenate([conv3,deconv3], axis = 3)


    deconv2 = Conv2DTranspose(128, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge3)
    
    merge2 = concatenate([conv2,deconv2], axis = 3)
    deconv1 = Conv2DTranspose(64, (w,w), padding='same', activation=tf.nn.relu, strides=(2,2),use_bias=UB)(merge2)
    merge1 = concatenate([conv1,deconv1], axis = 3)
    conv_last = Conv2D(1, (w,w), padding='same', activation=tf.nn.relu,use_bias=UB)(merge1)

    model = Model(inputs,conv_last)

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.00010), loss = "mean_squared_error", metrics = ["accuracy"])
    # model.compile(optimizer = 'adam', loss = "mean_squared_error", metrics = ["accuracy"])

    
    return model

model = create_model()

model.summary()


checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq = 250,
                                                 verbose = 1)

start_time = time.time()




Process_Dataset_path_gaussfilt = "/content/gdrive/MyDrive/MPhys Project/centroid/run_28/10^5/gaussfilt_process.mat"
X_val_blur = sio.loadmat(Process_Dataset_path_gaussfilt)
X_val_blur = X_val_blur['images_gaussfilt_mat']
X_val_blur = np.swapaxes(X_val_blur, 0, -1)
X_val_blur = np.swapaxes(X_val_blur, 1, 2)
X_val_blur = np.reshape(X_val_blur, (X_val_blur.shape[0], X_val_blur.shape[1], X_val_blur.shape[2], 1))

Process_Dataset_path_centroid = "/content/gdrive/MyDrive/MPhys Project/centroid/run_28/10^1/centroid_process.mat"
X_val = sio.loadmat(Process_Dataset_path_centroid)
X_val = X_val['images_sampled_mat']
X_val = np.swapaxes(X_val, 0, -1)
X_val = np.swapaxes(X_val, 1, 2)
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], X_val.shape[2], 1))

model.load_weights(checkpoint_path)
start_time = time.time()
predict = model.predict(X_val_blur)
testing_time = time.time()-start_time
dictionary = {}
dictionary['results'] = {'input':X_val_blur, 'predictions':predict, 'validation':X_val}
# dictionary['time'] = {'training_time': training_time, 'testing_time': testing_time}

sio.savemat("/content/gdrive/MyDrive/MPhys Project/centroid/run_28/10^1/results_process.mat", dictionary)
# sio.savemat("/content/gdrive/MyDrive/MPhys Project/run_15/history.mat", history.history)